In [1]:
import pandas as pd
from collections import Counter
from flair.models import SequenceTagger
from flair.data import Sentence
#from joblib import Parallel, delayed, cpu_count

In [2]:
# Load the preprocessed data
df = pd.read_csv('/Users/andy/Documents/DS/TFM SALLE/project/DjMario/DjMario_videos.csv')

# Combine all text columns into a single list
texts = df['title'].astype(str).tolist() + df['description'].astype(str).tolist() + df['transcript'].astype(str).tolist()

# Load the Flair NER model for Spanish
tagger = SequenceTagger.load("flair/ner-spanish-large")



2023-08-22 17:08:59,379 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [3]:
# list of names to recognize persons 
influencer_names = [
        'ibai',
        'auronplay',
        'auron',
        'marta díaz',
        'marta'
        'djmario',
        'djmariio',
        'clakovi',
        'elrubius',
        'rubius',
        'emma chamberlain',
        'emma',
        'chamberlain',
        'malbert',
        'mostopapi',
        'orslok',
        'vegetta',
        'vegeta']  

In [4]:
%%time
# Perform NER on text data and extract named entities
person_entities = []
for text in texts:
    sentence = Sentence(text)
    tagger.predict(sentence)
    for entity in sentence.get_spans('ner'):
        if entity.labels[0].value == 'PER':
            person_entities.append(entity.text)

    # Adding custom names detection
    for name in influencer_names:
        if name.lower() in text.lower():
            person_entities.append(name)



CPU times: user 9h 2min 51s, sys: 2h 46min 34s, total: 11h 49min 25s
Wall time: 3h 11min 9s


In [5]:
# Count the frequency of person entities
entity_frequencies = Counter(person_entities)

# Create a DataFrame and export to CSV
df_export = pd.DataFrame({
    'Source': ['YouTuber'] * len(entity_frequencies),
    'Target': list(entity_frequencies.keys()),
    'Weight': list(entity_frequencies.values())
})

df_export.to_csv('per_data.csv', index=False)